## 日出日落中天

In [1]:
import astropy.units as u
from astropy.coordinates import PrecessedGeocentric,get_body,solar_system_ephemeris,Angle
from astropy.time import Time,TimeDelta
import astropy.constants as const

import numpy as np

def obj_sight(date,objname,diskanglestr,lonstr,latstr,tzinhours):
    """
    date: string, date of observing. 
    objname : string, name of sun, moon or planet in solar system.
    diskanglestr : string, apparent angle of objname.
    lonstr : string, longitute of observing location on earth.
    latstr : string, latitute of observing location on earth.
    ltzinhours : float, time zone in hours of observing location.
    return : print out rise time, set time, transit time and cooresponding alt/az.
    # 台北(東經121.5654度，北緯25.032969度，時差+8小時)，標的物太陽(sun)的視角32角分。
    # Ex.1
    # 用下面指令列印出2018年1月1日在台北的日出、日落、中天時刻以及方位與仰角。
    obj_sight('2018-1-1','sun','0d32m0s','121.5654d','25.032969d',8) 
    """
    lon=Angle(lonstr)
    lat=Angle(latstr)
    halfdisk=Angle(diskanglestr)*0.5
    date=Time(Time(date,out_subfmt='date').iso)
    times=date+np.linspace(0.,1440.,1441)*TimeDelta(60,format='sec')  # 每分鐘計算位置
    deltat=tzinhours*TimeDelta(3600,format='sec')
    a=const.R_earth.to('m').value       # 地球半徑
    with solar_system_ephemeris.set('jpl'):
        n=len(times)
        gmttimes=times-deltat   # 世界時間 UTC
        # 標的物在及時赤道座標系統座標
        bodies=get_body(objname,gmttimes).transform_to(PrecessedGeocentric(equinox=gmttimes))  
        sides=gmttimes.sidereal_time('apparent', 'greenwich')+lon    # local sidereal time
        x=bodies.cartesian.x.to('m').value        # x,y,z為標的物在及時赤道座標系統的直角座標   
        y=bodies.cartesian.y.to('m').value
        z=bodies.cartesian.z.to('m').value
        px=np.array(a*np.cos(lat)*np.cos(sides))  # px,py,pz 為觀測地在及時赤道座標系統的直角坐標
        py=np.array(a*np.cos(lat)*np.sin(sides))
        pz=np.array(a*np.sin(lat)*np.ones(n))
    positions=np.vstack((px,py,pz)).T  # n x 3 array
    objs=np.vstack((x,y,z)).T          # n x 3 array
    # e_z 的每一行為天頂方向單位向量，e_n的每一行為地表往北單位向量，e_e的每一行為地表往東單位向量
    e_z=np.vstack((np.cos(lat)*np.cos(sides),np.cos(lat)*np.sin(sides),np.sin(lat)*np.ones(n)))  # 3 x n
    e_n=np.vstack((-np.sin(lat)*np.cos(sides),-np.sin(lat)*np.sin(sides),np.cos(lat)*np.ones(n))) # 3 x n
    e_e=np.vstack((-np.sin(sides),np.cos(sides),np.zeros(n)))                 # 3 x n
    coords=np.zeros((n,3))   # coords 的每一列代表標的物在地表座標系統的直角坐標
    az=np.zeros(n)           # az 標的物的方位角，北方為0，往東為正，正東方90度，正南方180度，正西方270度。
    alt=np.zeros(n)          # alt 標的物的仰角
    for i in range(n):
        coords[i]=np.array(np.vstack((e_e[:,i],e_n[:,i],e_z[:,i]))).dot(objs[i]-positions[i])
        r=np.sqrt(coords[i][0]**2+coords[i][1]**2)
        rho=np.sqrt(coords[i][0]**2+coords[i][1]**2+coords[i][2]**2)
        alt[i]=np.arcsin(coords[i][2]/rho)*180/np.pi
        az[i]=np.arctan2(coords[i][0],coords[i][1])*180/np.pi
        if az[i]<0:
            az[i]=az[i]+360
    thresh=-(halfdisk+Angle("0d34m0s")).deg   # 大氣折射，讓光線轉彎了34角分
    altdiff=alt-thresh        # altdif 變號的時候就是標的物升起或沉沒的區間，用內插進行估計
    for i in range(len(times)-1):
        if altdiff[i]*altdiff[i+1] <=0 :
            t=times[i]+(times[i+1]-times[i])*(0-altdiff[i])/(altdiff[i+1]-altdiff[i])
            if altdiff[i]<0:
                print("Rise",t,"Az",az[i])
            else:
                print("Set",t,"Az",az[i])
        if not (az[i] // 180 == az[i+1] // 180) and alt[i]>0:
            if np.abs(az[i]-180)<30:
                ratio=(180-az[i])/(az[i+1]-az[i])
                t=times[i]+(times[i+1]-times[i])*ratio
                print("Transit",t,"Alt",alt[i]+(alt[i+1]-alt[i])*ratio,"Az","South")
            else:
                left=az[i]
                right=az[i+1]
                if left>300:
                    left=left-360
                if right>300:
                    right=right-360
                ratio=(0-left)/(az[i+1]-az[i])
                t=times[i]+(times[i+1]-times[i])*ratio
                print("Transit",t,"Alt",alt[i]+(alt[i+1]-alt[i])*ratio,"Az","North")



In [2]:
# 台北(東經121.5654度，北緯25.032969度，時差+8小時)，標的物太陽(sun)的視角32角分。
# Ex.1
# 用下面指令列印出2018年1月1日在台北的日出、日落、中天時刻以及方位與仰角。
obj_sight('2018-1-1','sun','0d32m0s','121.5654d','25.032969d',8) 


Rise 2018-01-01 06:38:47.130 Az 115.066472753
Transit 2018-01-01 11:57:09.418 Alt 41.9575847767 Az South
Set 2018-01-01 17:15:36.479 Az 244.828654564


In [3]:
# Ex.2
obj_sight('2017-12-10','moon','0d32m0s','121.5654d','25.032969d',8)  


Transit 2017-12-10 05:38:22.471 Alt 73.5230263955 Az South
Set 2017-12-10 12:05:02.922 Az 278.440772592


In [4]:
# Ex.3
obj_sight('2017-12-4','moon','0d32m0s','121.5654d','25.032969d',8)


Set 2017-12-04 06:37:52.044 Az 290.09123571
Rise 2017-12-04 17:54:55.105 Az 68.7458437964


In [5]:
# 布宜諾斯艾利斯 Buenos Aires, Tz=-3
# https://www.timeanddate.com/sun/
# https://www.latlong.net/
obj_sight('2017-8-8','sun','0d32m0s','-58.3916d','-34.6037d',-3)

Rise 2017-08-08 07:40:36.120 Az 71.1062623927
Transit 2017-08-08 12:58:59.992 Alt 39.4426240526 Az North
Set 2017-08-08 18:18:06.110 Az 288.836186711


## EarthLocation的直角坐標與經緯度及高度的關係
地球上緯度$\delta$，經度$\alpha$，高度 $h$ 的地點，其直角坐標為何？

地球是扁橢球，其參考橢球是赤道半徑$a$,極半徑$b$的橢圓繞短軸旋轉出來的球體。

經線0度與經線180度合併為橢圓，其方程式為 $\frac{x^2}{a^2}+\frac{z^2}{b^2}+1$ ，參數式為 $x=a\cos\theta,z=b\sin\theta$。

在$(a\cos\theta,b\sin\theta)$切線斜率為 $-\frac{b}{a}\cot\theta$，而$\tan\delta$與斜率的乘積為 $-1$，所以 $\tan\delta=\frac{a}{b}\tan\theta$。因此 $\theta=\arctan(\frac{b}{a}\tan\delta)$

高度h在X軸,Z上的投影分別為$h\cos\delta,h\sin\delta$。所以經度0,緯度$\delta$,高度$h$的地方，
其直角坐標為 $$(a\cos\theta+h\cos\delta,0,b\sin\theta+h\sin\delta)。$$

所以經度$\alpha$,緯度$\delta$,高度$h$的地方，
$$((a\cos\theta+h \cos\delta)\cos\alpha,((a\cos\theta+h \cos\delta)\sin\alpha,b\sin\theta+h\sin\delta)，$$
其中$\theta=\arctan(\frac{b}{a}\tan(\delta))$


In [1]:
def location(lon,lat,h):
    """由經度lon緯度lat高度h，算地理的直角座標
    """
    a=6378137*u.m
    b=6356752.314245179*u.m
    theta=np.arctan(b/a*np.tan(lat))
    r1=a*np.cos(theta)+h*np.cos(lat)
    return np.array([(r1*np.cos(lon)).to('m').value,(r1*np.sin(lon)).to('m').value,(b*np.sin(theta)+h*np.sin(lat)).to('m').value])

In [3]:
import astropy.constants as const
import astropy.units as u
from astropy.coordinates import EarthLocation
import numpy as np
lon=121.55*u.deg
lat=24.99*u.deg
h=1097*u.m
loc=EarthLocation.from_geodetic(lat=lat,lon=lon,height=h)
print("赤道",EarthLocation.from_geodetic(lat=0*u.deg,lon=0*u.deg))
print("北極",EarthLocation.from_geodetic(lat=90*u.deg,lon=0*u.deg))
pos=np.array(loc.to('m').value)
print("台北七星山，Astropy",pos)
pos2=location(lon,lat,h)
print("台北七星山，location",pos2)
print("差距",pos-pos2)
#print([loc.value[i] for i in range(3)])

赤道 (6378137.0, 0.0, 0.0) m
北極 (3.9186209248144716e-10, 0.0, 6356752.314245179) m
台北七星山，Astropy [-3027206.79534348  4930289.75284083  2678533.91819335]
台北七星山，location [-3027206.79534348  4930289.75284083  2678533.91819335]
差距 [ 0.  0.  0.]
